In [35]:
import torch
import pandas as pd
from torch import nn

In [2]:
def load_tensor(filepath):
    tensor = torch.load(filepath)
    return tensor

In [4]:
root_dir = '/import/network-temp/yimengg/data/'
base_model = 'blip-2'
few_shot_topic = 'military'
epoch = 9

In [ ]:
test_pred_few_shot_on_military = load_tensor(root_dir + f'twitter-comms/processed_data/tensor/{base_model}_test_pred_fs_{few_shot_topic}_epoch_{epoch}.pt')
test_true = load_tensor(root_dir + f'twitter-comms/processed_data/tensor/{base_model}_test_true_fs_{few_shot_topic}_epoch_{epoch}.pt')

assert test_pred_few_shot_on_military.shape == torch.Size([22082, 2])
assert test_true.shape == torch.Size([22082])

test_pred_full_train = load_tensor(root_dir + f'twitter-comms/processed_data/tensor/{base_model}_test_pred_fs_None_epoch_{epoch}.pt')

In [11]:
feather_path = '../raw_data/val_completed_exist.feather'

df = pd.read_feather(feather_path)

In [ ]:
df['is_military'] = df['topic'].apply(lambda topic: 'military' in topic)
row_excluded = df[df["is_military"] == True].index

In [86]:
print(row_excluded)   # elements in row_excluded are indices of the dataframe

Int64Index([14944, 14945, 14946, 14947, 14948, 14949, 14950, 14951, 14952,
            14953,
            ...
            22072, 22073, 22074, 22075, 22076, 22077, 22078, 22079, 22080,
            22081],
           dtype='int64', length=7138)


In [34]:
idx = 14952
print("few shot on military:")
print(test_pred_few_shot_on_military[row_excluded[20:30], :])
print("full train:")
print(test_pred_full_train[row_excluded[20:30], :])
print("ground true:")
print(test_true[row_excluded[20:30]])

few shot on military:
tensor([[0.9401, 0.0599],
        [0.9401, 0.0599],
        [0.0844, 0.9156],
        [0.8804, 0.1196],
        [0.9571, 0.0429],
        [0.9252, 0.0748],
        [0.9252, 0.0748],
        [0.4426, 0.5574],
        [0.5611, 0.4389],
        [0.9801, 0.0199]], device='cuda:0')
full train:
tensor([[0.5814, 0.4186],
        [0.5814, 0.4186],
        [0.0624, 0.9376],
        [0.4702, 0.5298],
        [0.6837, 0.3163],
        [0.6671, 0.3329],
        [0.6671, 0.3329],
        [0.3372, 0.6628],
        [0.2761, 0.7239],
        [0.8873, 0.1127]], device='cuda:0')
ground true:
tensor([0, 0, 1, 1, 1, 0, 0, 1, 1, 0])


In [40]:
label_pred_few_shot_on_military = torch.zeros_like(test_true)

label_pred_few_shot_on_military[test_pred_few_shot_on_military[:, 1] >= 0.5] = 1

In [53]:
misclassified_indices_few_shot_on_military = (((label_pred_few_shot_on_military[row_excluded] == test_true[row_excluded])==False).nonzero(as_tuple=True)[0])

In [50]:
label_pred_full_train = torch.zeros_like(test_true)

label_pred_full_train[test_pred_full_train[:, 1] >= 0.5] = 1

In [54]:
misclassified_indices_full_train = (((label_pred_full_train[row_excluded] == test_true[row_excluded])==False).nonzero(as_tuple=True)[0])

In [ ]:
misclassified_indices_set_few_shot_on_military = set(misclassified_indices_few_shot_on_military.tolist())   # 1961

In [64]:
print(len(misclassified_indices_set_few_shot_on_military))

1961


In [61]:
misclassified_indices_set_full_train = set(misclassified_indices_full_train.tolist())   # 1367

In [65]:
print(len(misclassified_indices_set_full_train))

1367


In [70]:
few_shot_diff = misclassified_indices_set_few_shot_on_military.difference(misclassified_indices_set_full_train)   # 1052

In [71]:
union = misclassified_indices_set_few_shot_on_military.union(misclassified_indices_set_full_train)   # 2419

In [72]:
intersection = misclassified_indices_set_few_shot_on_military.intersection(misclassified_indices_set_full_train)   # 909

In [75]:
torch.tensor(list(few_shot_diff))

tensor([2048, 4097,    4,  ..., 4092, 4093, 2047])

In [80]:
idx = 4093
print("few shot on military:")
print(test_pred_few_shot_on_military[row_excluded[idx], :])
print("full train:")
print(test_pred_full_train[row_excluded[idx], :])
print("ground true:")
print(test_true[row_excluded[idx]])

few shot on military:
tensor([0.8388, 0.1612], device='cuda:0')
full train:
tensor([0.3583, 0.6417], device='cuda:0')
ground true:
tensor(1)


In [83]:
df.iloc[row_excluded[idx]]

id                                           1277028418719232001
full_text      US fighter jets intercept Russian military air...
image_id                                     1027023012372066304
filename          69/1027023012372066304-1027023010429984769.jpg
falsified                                                   True
topic                                              military_hard
exists                                                      True
is_military                                                 True
Name: 19037, dtype: object